<a href="https://colab.research.google.com/github/FaustinoSachimuco/Mineracao_de_Dados/blob/Faustino_Testes/Ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Dados de exemplo
"""textos = [
    "Aliança Democrática (AD) é uma coligação política em Portugal de centro-direita, formada pelo Partido Social Democrata (PPD/PSD), pelo CDS – Partido Popular (CDS–PP), pelo Partido Popular Monárquico (PPM) e personalidades independentes.[1][2] Teve como fundadores Luís Montenegro, Nuno Melo e Gonçalo da Câmara Pereira, bem como diversas personalidades independentes ",
    "O gato dormiu no sofá.",
    "O pássaro voou alto no céu."
]
"""

# Função para ler o arquivo de texto e retornar o conteúdo como uma lista de strings
def ler_arquivo(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.readlines()
    return text

# Caminho para o arquivo de texto
file_path = 'ad.txt'

# Ler o conteúdo do arquivo
textos = ler_arquivo(file_path)


# Tokenização dos textos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(textos)
total_palavras = len(tokenizer.word_index) + 1

# Criando sequências de tokens
sequencias = []
for texto in textos:
    sequencia = tokenizer.texts_to_sequences([texto])[0]
    for i in range(1, len(sequencia)):
        n_gram_sequence = sequencia[:i+1]
        sequencias.append(n_gram_sequence)

# Padding das sequências para garantir que tenham o mesmo comprimento
max_len = max([len(x) for x in sequencias])
sequencias_padded = pad_sequences(sequencias, maxlen=max_len, padding='pre')

# Dividindo as sequências em X e y
X = np.array(sequencias_padded[:,:-1])
y = np.array(sequencias_padded[:,-1])

# Convertendo y para one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_palavras)

# Modelo de Linguagem Simples
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_palavras, 100, input_length=max_len-1),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_palavras, activation='softmax')
])

modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinamento do modelo
modelo.fit(X, y, epochs=100, verbose=2)

# Função para gerar texto usando o modelo treinado
def gerar_texto(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(modelo.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Exemplo de uso
print(gerar_texto("O", 5, max_len))



Epoch 1/100
1615/1615 - 21s - loss: 6.8861 - accuracy: 0.0679 - 21s/epoch - 13ms/step
Epoch 2/100
1615/1615 - 12s - loss: 6.1794 - accuracy: 0.1085 - 12s/epoch - 7ms/step
Epoch 3/100
1615/1615 - 11s - loss: 5.7369 - accuracy: 0.1415 - 11s/epoch - 7ms/step
Epoch 4/100
1615/1615 - 11s - loss: 5.3614 - accuracy: 0.1600 - 11s/epoch - 7ms/step
Epoch 5/100
1615/1615 - 11s - loss: 5.0214 - accuracy: 0.1768 - 11s/epoch - 7ms/step
Epoch 6/100
1615/1615 - 11s - loss: 4.7106 - accuracy: 0.1942 - 11s/epoch - 7ms/step
Epoch 7/100
1615/1615 - 11s - loss: 4.4190 - accuracy: 0.2137 - 11s/epoch - 7ms/step
Epoch 8/100
1615/1615 - 11s - loss: 4.1379 - accuracy: 0.2360 - 11s/epoch - 7ms/step
Epoch 9/100
1615/1615 - 11s - loss: 3.8661 - accuracy: 0.2631 - 11s/epoch - 7ms/step
Epoch 10/100
1615/1615 - 11s - loss: 3.6034 - accuracy: 0.2945 - 11s/epoch - 7ms/step
Epoch 11/100
1615/1615 - 11s - loss: 3.3569 - accuracy: 0.3285 - 11s/epoch - 7ms/step
Epoch 12/100
1615/1615 - 11s - loss: 3.1221 - accuracy: 0.3660

In [9]:
print(gerar_texto("objetivo da alinça democratica ", 35, max_len))

1/1 [==============================] - 0s 18ms/step
objetivo da alinça democratica  inovação e a produção de mobilidade social através de uma ampla e alargada reflexão a es pública das “entre ao profissiona política comum de instalação de re comercial emigrantes que com uma resposta de pro
